In [ ]:
import akshare as ak
import pandas as pd

In [ ]:
symbol_id = "000001"
stock_record = ak.stock_zh_a_hist(symbol=symbol_id, adjust="hfq")

stock_record.to_csv("{}.csv".format(symbol_id))

In [ ]:
stock_record = pd.read_csv(f"{symbol_id}.csv", index_col='日期', parse_dates=["日期"])[['开盘', '收盘', '最高', '最低']]
# 修改索引和column的名字
stock_record.index.name = 'date'
stock_record.columns = ['open', 'close', 'high', 'low']

In [ ]:
# 收盘比开盘上涨3%及以上的日期
stock_record[(stock_record['close'] - stock_record['open'])/stock_record['open'] >= 0.03].index

In [ ]:
# 开盘比前日收盘跌2%及以上的日期
stock_record[(stock_record['open'] - stock_record['close'].shift(1))/stock_record['close'].shift(1) <= -0.02].index

In [ ]:
# 从2010年1月1日开始，每月第一个交易日买入1手，每年最后一个交易日卖出所有，到今天的收益
